## * In this notebook I would like to select, manipulate, and transform the existing features into valuable features that can be used for my further modeling.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import precision_score

In [2]:
orders = pd.read_csv(r'orders.csv') #Null values in days_since_prior_order
products = pd.read_csv(r'products.csv')
aisles = pd.read_csv(r'aisles.csv')
departments = pd.read_csv(r'departments.csv')
order_products_prior = pd.read_csv(r'order_products__prior.csv')
sample_submission = pd.read_csv(r'sample_submission.csv')
order_products_train = pd.read_csv(r'order_products__train.csv')

## Data cleaning

In [3]:
# Get the statistics of your dataframe
departments.describe()
products.describe()
order_products_prior.describe()
aisles.describe()
orders.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [4]:
missing = orders.isnull().sum()
Percent_missing = (missing/len(orders))*100
Percent_missing

order_id                  0.000000
user_id                   0.000000
eval_set                  0.000000
order_number              0.000000
order_dow                 0.000000
order_hour_of_day         0.000000
days_since_prior_order    6.027594
dtype: float64

From above we observed that, only 6% of days_since_prior_order column is null. So we decided to truncate them

----------

In [5]:
orders=orders[orders['days_since_prior_order'].notnull()]
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0


In [6]:
#checking missing value for departments
departments.isnull().any()
#no missisng value

department_id    False
department       False
dtype: bool

In [7]:
#checking missing value for aisles
aisles.isnull().any()
#no missisng value

aisle_id    False
aisle       False
dtype: bool

In [8]:
#checking missing value for products
products.isnull().any()
#no missisng value

product_id       False
product_name     False
aisle_id         False
department_id    False
dtype: bool

In [9]:
#checking missing value for order_products_prior
order_products_prior.isnull().any()
#no missisng value

order_id             False
product_id           False
add_to_cart_order    False
reordered            False
dtype: bool

In [10]:
#checking missing value for order_products_prior
order_products_train.isnull().any()
#no missisng value

order_id             False
product_id           False
add_to_cart_order    False
reordered            False
dtype: bool

There aren't any missing value except for orders data which we have handled.


60% of the products are re-ordered from the train dataset

----------

### Merging

In [11]:
merged1 = pd.merge(order_products_train, orders, on='order_id', how='left')
merged1.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0
1,1,11109,2,1,112108,train,4,4,10,9.0
2,1,10246,3,0,112108,train,4,4,10,9.0
3,1,49683,4,0,112108,train,4,4,10,9.0
4,1,43633,5,1,112108,train,4,4,10,9.0


In [12]:
df_merged1 = pd.merge(merged1, products, on='product_id', how='left')
df_merged1.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16
1,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16
2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4
3,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4
4,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15


In [13]:
#Final data set is generated using all datasets
df = pd.merge(df_merged1, departments, on='department_id', how='left')
df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs
1,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs
2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,produce
3,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,produce
4,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods


In [14]:
df_new = df.copy()

In [15]:

del df['eval_set']

del df['add_to_cart_order']

In [16]:
df.head()

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department
0,1,49302,1,112108,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs
1,1,11109,1,112108,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs
2,1,10246,0,112108,4,4,10,9.0,Organic Celery Hearts,83,4,produce
3,1,49683,0,112108,4,4,10,9.0,Cucumber Kirby,83,4,produce
4,1,43633,1,112108,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods


In [17]:
#Getting average days between orders as a feature by using days_since_prior_order
df['avg_days_btwn_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
df['avg_days_btwn_orders'] = df['avg_days_btwn_orders'].replace(np.nan, 0)

In [18]:
df['avg_days_btwn_orders'] = df['avg_days_btwn_orders'].replace(0, 1)

In [19]:
#number of orders as a feature using the orders dataset
df['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)
df['nb_orders'] = df['nb_orders'].replace(np.nan, 0)

In [20]:
df.head()

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department,avg_days_btwn_orders,nb_orders
0,1,49302,1,112108,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs,1.000000,0.0
1,1,11109,1,112108,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,19.000000,10.0
2,1,10246,0,112108,4,4,10,9.0,Organic Celery Hearts,83,4,produce,16.285715,14.0
3,1,49683,0,112108,4,4,10,9.0,Cucumber Kirby,83,4,produce,12.000000,12.0
4,1,43633,1,112108,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,17.000000,5.0


In [21]:
#Getting the total items using the entire dataset 
df['total_items'] = df_merged1.groupby('user_id').size().astype(np.int16)
df['total_items'] = df['total_items'].replace(np.nan, 0)
df.head(5)

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department,avg_days_btwn_orders,nb_orders,total_items
0,1,49302,1,112108,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs,1.000000,0.0,0.0
1,1,11109,1,112108,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,19.000000,10.0,11.0
2,1,10246,0,112108,4,4,10,9.0,Organic Celery Hearts,83,4,produce,16.285715,14.0,31.0
3,1,49683,0,112108,4,4,10,9.0,Cucumber Kirby,83,4,produce,12.000000,12.0,0.0
4,1,43633,1,112108,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,17.000000,5.0,0.0


In [22]:
#getting average basket as a feature by using total items and number of orders
df['average_basket'] = (df.total_items /df.nb_orders).astype(np.float32)
df.head()

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department,avg_days_btwn_orders,nb_orders,total_items,average_basket
0,1,49302,1,112108,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs,1.000000,0.0,0.0,NaN
1,1,11109,1,112108,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,19.000000,10.0,11.0,1.100000
2,1,10246,0,112108,4,4,10,9.0,Organic Celery Hearts,83,4,produce,16.285715,14.0,31.0,2.214286
3,1,49683,0,112108,4,4,10,9.0,Cucumber Kirby,83,4,produce,12.000000,12.0,0.0,0.000000
4,1,43633,1,112108,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,17.000000,5.0,0.0,0.000000


In [23]:
df['average_basket'] = df['average_basket'].replace(np.nan, 0)

In [24]:
# creating a days_since_ratio using days_since_prior_order and average_days_between_orders
df['days_since_ratio'] = df.days_since_prior_order / df.avg_days_btwn_orders
df.head()

,order_id,product_id,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department,avg_days_btwn_orders,nb_orders,total_items,average_basket,days_since_ratio
0,1,49302,1,112108,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs,1.000000,0.0,0.0,0.000000,9.000000
1,1,11109,1,112108,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,19.000000,10.0,11.0,1.100000,0.473684
2,1,10246,0,112108,4,4,10,9.0,Organic Celery Hearts,83,4,produce,16.285715,14.0,31.0,2.214286,0.552632
3,1,49683,0,112108,4,4,10,9.0,Cucumber Kirby,83,4,produce,12.000000,12.0,0.0,0.000000,0.750000
4,1,43633,1,112108,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,17.000000,5.0,0.0,0.000000,0.529412


In [25]:
del df['user_id']
del df['product_name']
del df['department']
df.head()

,order_id,product_id,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,avg_days_btwn_orders,nb_orders,total_items,average_basket,days_since_ratio
0,1,49302,1,4,4,10,9.0,120,16,1.000000,0.0,0.0,0.000000,9.000000
1,1,11109,1,4,4,10,9.0,108,16,19.000000,10.0,11.0,1.100000,0.473684
2,1,10246,0,4,4,10,9.0,83,4,16.285715,14.0,31.0,2.214286,0.552632
3,1,49683,0,4,4,10,9.0,83,4,12.000000,12.0,0.0,0.000000,0.750000
4,1,43633,1,4,4,10,9.0,95,15,17.000000,5.0,0.0,0.000000,0.529412


Converting hour, aisle, dept, product, days_since_prior_order, day of week into categories.

In [26]:
hour = {c:i for i,c in enumerate(df['order_hour_of_day'].unique())}
aisle = {c:i for i,c in enumerate(df['aisle_id'].unique())}
dept = {c:i for i,c in enumerate(df['department_id'].unique())}
product = {c:i for i,c in enumerate(df['product_id'].unique())}

In [27]:
df['aisle_new'] = [float(aisle[t]) for t in df.aisle_id]

df['dept_new'] = [float(dept[t]) for t in df.department_id]
df['product_new'] = [float(product[t]) for t in df.product_id]

df['order_hour_of_day_new'] = [hour[t] for t in df.order_hour_of_day]

In [28]:
df['order_hour_of_day_new'].value_counts()

5     119370
11    116198
6     114762
15    114119
8     111752
0     110479
2     110237
9      96944
10     93856
1      76522
4      67386
13     58175
3      40920
12     36302
7      34813
16     27319
17     16965
14     11847
18      9083
20      5626
22      3847
21      3226
23      2438
19      2431
Name: order_hour_of_day_new, dtype: int64

In [29]:
dow = {c:i for i,c in enumerate(df['order_dow'].unique())}
df['order_dow_new'] = [float(dow[t]) for t in df.order_dow]
dspo = {c:i for i,c in enumerate(df['days_since_prior_order'].unique())}
df['days_since_prior_order__new'] = [float(dspo[t]) for t in df.days_since_prior_order]
df['reordered'] = df['reordered'].astype('float')

In [30]:
df.head()

,order_id,product_id,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id,avg_days_btwn_orders,nb_orders,total_items,average_basket,days_since_ratio,aisle_new,dept_new,product_new,order_hour_of_day_new,order_dow_new,days_since_prior_order__new
0,1,49302,1.0,4,4,10,9.0,120,16,1.000000,0.0,0.0,0.000000,9.000000,0.0,0.0,0.0,0,0.0,0.0
1,1,11109,1.0,4,4,10,9.0,108,16,19.000000,10.0,11.0,1.100000,0.473684,1.0,0.0,1.0,0,0.0,0.0
2,1,10246,0.0,4,4,10,9.0,83,4,16.285715,14.0,31.0,2.214286,0.552632,2.0,1.0,2.0,0,0.0,0.0
3,1,49683,0.0,4,4,10,9.0,83,4,12.000000,12.0,0.0,0.000000,0.750000,2.0,1.0,3.0,0,0.0,0.0
4,1,43633,1.0,4,4,10,9.0,95,15,17.000000,5.0,0.0,0.000000,0.529412,3.0,2.0,4.0,0,0.0,0.0


In [31]:
del df['aisle_id']

del df['department_id']

In [32]:
del df['order_hour_of_day']
del df['order_dow']
del df['days_since_prior_order']
del df['product_id']

In [33]:
# creating new features such as orders,reorders and reorder_rate for predicting the department variable

df_new['orders'] = df_new.groupby(df_new.product_id).size().astype(np.int32) 
df_new['orders'] = df_new['orders'].replace(np.nan,0)
df_new['reorders'] = df_new['reordered'].groupby(df_new.product_id).sum().astype(np.float32)
df_new['reorders'] = df_new['reorders'].replace(np.nan,0)
df_new['reorder_rate'] = (df_new.reorders / df_new.orders).astype(np.float32)
df_new['reorder_rate'] = df_new['reorder_rate'].replace(np.nan,0)

In [34]:
df_new['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
df_new['average_days_between_orders'] = df_new['average_days_between_orders'].replace(np.nan, 0)

In [35]:
df_new['average_days_between_orders'] = df_new['average_days_between_orders'].replace(0, 1)

In [36]:
df_new['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)
df_new['nb_orders'] = df_new['nb_orders'].replace(np.nan, 0)

In [37]:
df_new['total_items'] = df_merged1.groupby('user_id').size().astype(np.int16)
df_new['total_items'] = df_new['total_items'].replace(np.nan, 0)

In [38]:
df_new['average_basket'] = (df_new.total_items /df_new.nb_orders).astype(np.float32)

In [39]:
df_new.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,...,aisle_id,department_id,department,orders,reorders,reorder_rate,average_days_between_orders,nb_orders,total_items,average_basket
0,1,49302,1,1,112108,train,4,4,10,9.0,...,120,16,dairy eggs,0.0,0.0,0.000000,1.000000,0.0,0.0,NaN
1,1,11109,2,1,112108,train,4,4,10,9.0,...,108,16,dairy eggs,76.0,49.0,0.644737,19.000000,10.0,11.0,1.100000
2,1,10246,3,0,112108,train,4,4,10,9.0,...,83,4,produce,4.0,1.0,0.250000,16.285715,14.0,31.0,2.214286
3,1,49683,4,0,112108,train,4,4,10,9.0,...,83,4,produce,6.0,6.0,1.000000,12.000000,12.0,0.0,0.000000
4,1,43633,5,1,112108,train,4,4,10,9.0,...,95,15,canned goods,22.0,14.0,0.636364,17.000000,5.0,0.0,0.000000


In [40]:
df_new['average_basket'] = df_new['average_basket'].replace(np.nan, 0)
df_new['days_since_ratio'] = df_new.days_since_prior_order / df_new.average_days_between_orders
df_new['order_hour_of_day_new'] = [float(hour[t]) for t in df_new.order_hour_of_day]
df_new['reordered'] = df_new['reordered'].astype('float')
df_new['order_dow_new'] = [float(dow[t]) for t in df_new.order_dow]
df_new['days_since_prior_order__new'] = [float(dspo[t]) for t in df_new.days_since_prior_order]
df_new['dept_new'] = [float(dept[t]) for t in df_new.department_id]
df_new['product_new'] = [float(product[t]) for t in df_new.product_id]

In [41]:

del df_new['days_since_prior_order']
del df_new['order_dow']
del df_new['order_hour_of_day']
del df_new['department_id']
del df_new['aisle_id']
del df_new['product_id']
del df_new['add_to_cart_order']
del df_new['eval_set']
del df_new['department']
del df_new['product_name']
del df_new['product_new']

## Saving the data into pickle files

In [42]:
import pickle
df.to_pickle("./feature_dataset.pkl")
df_new.to_pickle("./dataset.pkl")